In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import os
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
#from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
import random

In [2]:
class Webcrawler(object):
    '''
    Class to implement webcrawler
    '''
#setting up the website to avoid the 'Access denied' issue    
    def website_setup(self):
        options = webdriver.ChromeOptions()
        options.add_argument('--window-size=1036,674')
        options.add_argument('--disable-blink-features=AutomationControlled')
        preferences = {"download.default_directory" : os.getcwd()}
        options.add_experimental_option("prefs",preferences)
        driver=webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        return driver
#getting data from the national stock exchange website    
    def get_data(self,symbol,driver,start_date,end_date):
        url = 'https://www.nseindia.com/get-quotes/equity?symbol='
        #concatenating the url with url with the stock passed as parameter
        new_url = url + symbol
        driver.get(new_url)
        #introducing delay to load the website before executing next command
        time.sleep(5)
        #adding the below command to avoid webdriver.web element not found exception
     #   driver.execute_script("window.scrollBy(0,800)","")
        #finding the historical data page on nse website to retrieve past stock's data
        historical_data_page = driver.find_element('link text','Historical Data')
        time.sleep(5)
        historical_data_page.click()
        time.sleep(5)
        '''
        selecting the range as 7 days to provide the data for last 7 days.
        '''
#         historic_bt=self.webdriver.find_element(By.LINK_TEXT,value="Historical Data")
        driver.execute_script("arguments[0].click();",historical_data_page)
#         select_range = driver.find_element(By.XPATH,'//*[@id="historical-trade"]/section/div/div[1]/div/div[1]/ul/li[2]/a')
#         from_date = driver.find_element(By.XPath,'//*[@id="startDate1"]')
#         to_date = driver.find_element(By.XPath,'//*[@id="endDate1"]')
#         start = input("")
#         end = input("")
        
        start_date_argument_set = f"arguments[0].value='{start_date}'"
        s_date=driver.find_element(By.ID,"startDate1")
        driver.execute_script(start_date_argument_set,s_date)
        
        end_date_argument_set = f"arguments[0].value='{end_date}'"
        e_date=driver.find_element(By.ID,"endDate1")
        driver.execute_script(end_date_argument_set,e_date)
        
        
        equity_button=driver.find_element(By.ID,"equity-historical-Date-filter")
        driver.execute_script("arguments[0].click()",equity_button)
        
        #select date
        time.sleep(5)
        #click on dropdown
        dropdown_button=driver.find_element(By.XPATH,"//button[@class='multiselect dropdown-toggle btn btn-default']")
        driver.execute_script("arguments[0].click()",dropdown_button)
        
        time.sleep(5)
        #click on eq
        eq=driver.find_element(By.XPATH,"//input[@value='EQ']")
        driver.execute_script("arguments[0].click()",eq)
        time.sleep(5)
        
        
        
        
        #select date
        click_filter_button=driver.find_element(By.XPATH,'//*[@id="equity-historical-Date-filter"]/div[3]/button')
#         driver.execute_script("arguments[0].click()",click_filter_button)
        click_filter_button.click()
        time.sleep(5)
        #accessinh the stock's table on historical data page to retrieve stock data
        table_content = driver.find_element(By.XPATH,'//*[@id="equityHistoricalTable"]/tbody')
        time.sleep(5)
        table_html = table_content.get_attribute('innerHTML')
        time.sleep(5)
        table_html = BeautifulSoup(table_html, 'html.parser')
        time.sleep(5)
        data = []
        #storing the table's data in a list
        for item in table_html.find_all('td'):
            data.append(item.text)
        driver.quit()
        return data

    
    


In [ ]:




class Node(object):
    '''
    Class to implement node
    '''

    def __init__(self, key, level):
        self.key = key

        # list to hold references to node of different level
        self.right = [None] * (level + 1)


class SkipList(object):
    '''
    Class to implement Skip list
    '''

    def __init__(self, max_lvl, P):
        # Maximum level for this skip list
        self.MAXLVL = max_lvl

        # P is the fraction of the nodes with level
        # i references also having level i+1 references
        self.P = P

        # create header node and initialize key to -1
        self.header = self.createNode(self.MAXLVL, -1)

        # current level of skip list
        self.level = 0
        self.c = 0

    # create  new node
    def createNode(self, lvl, key):
        n = Node(key, lvl)
        return n
 
    # create random level for node
    def randomLevel(self):
        lvl = 0
        while random.random() < self.P and \
                lvl < self.MAXLVL: lvl += 1
        return lvl

    # insert given key in skip list
    def insertElement(self, key):
        # create update array and initialize it
        self.c = self.c + 1 
        update = [None] * (self.MAXLVL + 1)
        current = self.header

        '''
        insertion in skip list by saving the reference to update list and 
        using the right list to traverse to lower levels and next nodes
        '''
        for i in range(self.level, -1, -1):
            while True:
                if(current.right[i] == None or current.right[i].key >= key): 
                    break
                current = current.right[i]
            update[i] = current

        ''' 
        reached level 0 and right list reference to 
        right, which is desired position to 
        insert key.
        '''
        current = current.right[0]

        '''
        if current is NULL that means we have reached
           to end of the level or current's key is not equal
           to key to insert that means we have to insert
           node between update[0] and current node
       '''
        if current == None or current.key != key:
            # Generate a random level for node
            rlevel = self.randomLevel()

            '''
            If random level is greater than list's current
            level (node with highest level inserted in 
            list so far), initialize update value with reference
            to header for further use
            '''
            if rlevel > self.level:
                for i in range(self.level + 1, rlevel + 1):
                    update[i] = self.header
                self.level = rlevel

            # create new node with random level generated
            n = self.createNode(rlevel, key)

            # insert node by rearranging references
            for i in range(rlevel + 1):
                n.right[i] = update[i].right[i]
                update[i].right[i] = n

            print("Successfully inserted key {}".format(key))

    def deleteElement(self, search_key):

        # create update array and initialize it
        update = [None] * (self.MAXLVL + 1)
        current = self.header

        '''
        start from highest level of skip list
        move the current reference right while key 
        is greater than key of node next to current
        Otherwise inserted current in update and 
        move one level down and continue search
        '''
        for i in range(self.level, -1, -1):
            while (current.right[i] and current.right[i].key < search_key):
                current = current.right[i]
            update[i] = current

        ''' 
        reached level 0 and advance reference to 
        right, which is possibly our desired node
        '''
        current = current.right[0]

        # If current node is target node
        if current != None and current.key == search_key:

            '''
            start from lowest level and rearrange references 
            just like we do in singly linked list
            to remove target node
            '''
            for i in range(self.level + 1):

                '''
                If at level i, next node is not target 
                node, break the loop, no need to move 
                further level
                '''
                if update[i].right[i] != current:
                    break
                update[i].right[i] = current.right[i]

            # Remove levels having no elements
            while (self.level > 0 and self.header.right[self.level] == None):
                self.level -= 1
            print("Successfully deleted the stock {}".format(search_key))

    def searchElement(self, key):
        current = self.header

        for i in range(self.level, -1, -1):
            while (current.right[i] and current.right[i].key < key):
                current = current.right[i]

        # reached level 0 and advance reference to
        # right, which is prssibly our desired node
        current = current.right[0]

        # If current node have key equal to
        # search key, we have found our target node
        if current and current.key == key:
         #   print("Found Stock {} ", key)
            return True
        else :
          #  print("Stock not found")
            return False
        return False

    # Display skip list level wise
    def displayList(self):
        #print("\n*****Skip List******")
        head = self.header
        for lvl in range(self.level + 1):
            print("Level {}: ".format(lvl), end=" ")
            node = head.right[lvl]
            while (node != None):
                print(node.key, end=" ")
                node = node.right[lvl]
            print("")
    def kthminimum(self,k):
        head = self.header
        node = head.right[0]
        count = 0
        while(count < k-1):
            node = node.right[0]
            count += 1
        return node.key 
     
    def kthmaximum(self,k):
        head = self.header
        node = head.right[0]
        count = 0
        r = self.c 
        while(count < r-k-1):
            node = node.right[0]
            count += 1
        return node.key          
#function for intersection of skiplist            
def intersection(data1,data2,skplst1,skplst2):
    skplst3 = SkipList(3, 0.5)
    if(len(data1) <= len(data2)):
        for i in range(5,len(data1),14):
           data1[i] = data1[i].replace(",","") 
           if(skplst2.searchElement(float(data1[i]))):
                skplist3.insertElement(data1[i])
    else:
        for i in range(5,len(data2),14):
           data2[i] = data2[i].replace(",","") 
           if(skplst1.searchElement(float(data2[i]))):
                skplist3.insertElement(float(data2[i]))
    print("Skiplist after intersection: " )
    skplst3.displayList() 
#    return skplst3
                
    
    
        
        
    
# Driver to test above code
def main():
#     symbol = input("Enter the stock to be searched")
    symbol1 = 'INFY'
    symbol2 = 'TITAN'
    #calling the Webcrawler objects 
    crawler1 = Webcrawler()
    crawler2 = Webcrawler()
    
    # getting the webdriver object
    driver1 = crawler1.website_setup()
    # getting the desired data from webcrawler object
    data1 = crawler1.get_data(symbol1,driver1,'8-10-2022','18-10-2022')
    
    # getting the webdriver object
    driver2 = crawler2.website_setup()
    # getting the desired data from webcrawler object
    data2 = crawler2.get_data(symbol2,driver2,'8-10-2022','18-10-2022')
    
    
    
    # calling the skiplist object
    skplst1 = SkipList(3, 0.5)
    skplst2 = SkipList(3, 0.5)
    skplst3 = SkipList(3, 0.5)
    
    #inserting stock data scraped by  webcrawler in skiplist
    for i in range(5,len(data1),14):
        data1[i] = data1[i].replace(",","")
        skplst1.insertElement(float(data1[i]))
    print("-----Skiplist for stock 'INFY'-------")    
    skplst1.displayList()
    for i in range(5,len(data2),14):
        data2[i] = data2[i].replace(",","")
        skplst2.insertElement(float(data2[i]))
    print("-----Skiplist for stock 'TITAN'-------") 
    skplst2.displayList()
    intersection(data1,data2,skplst1,skplst2)
    
    # Enter the value to be searched and then the value is searched in SkipList
    k = float(input("Enter the element to be searched in skiplist1 for the stock 'INFY': "))
    if(skplst1.searchElement(k)):
        print("Found stock for 'INFY' {}".format(k))
    else :
        print("Stock not found")
        
     # Enter the value to be searched and then the value is searched in SkipList 
    r = float(input("Enter the element to be searched IN skiplist2 for the stock 'Titan' "))
    if(skplst2.searchElement(r)):
        print("Found stock for 'TITAN' {}".format(r))
    else :
        print("Stock not found")
    
    #function for intersection of skiplist1 and skiplist2 to find common stock prices        
   
    # Enter the value to be searched and then the value is deleted in SkipList
    m = float(input("Enter the element to be deleted in skiplist2 for the stock 'INFY':: "))
    skplst1.deleteElement(m)
    print("Skiplist1 after deletion is: ") 
    skplst1.displayList()
    n = float(input("Enter the element to be deleted in skiplist2 for the stock 'Titan':: "))
    skplst2.deleteElement(n)
    print("Skiplist2 after deletion is: ") 
    skplst2.displayList()
    
    k = int(input("Enter the kth position"))
    print("{} minimum element is:".format(k))
    print(skplst1.kthminimum(k))
    print("{} maximum element is:".format(k))
    print(skplst2.kthmaximum(k))
    
    
            

main()

Successfully inserted key 1474.25
Successfully inserted key 1419.9
Successfully inserted key 1428.7
Successfully inserted key 1423.8
Successfully inserted key 1462.65
Successfully inserted key 1451.2
-----Skiplist for stock 'INFY'-------
Level 0:  1419.9 1423.8 1428.7 1451.2 1462.65 1474.25 
Level 1:  1419.9 1423.8 1428.7 1451.2 1462.65 
Level 2:  1419.9 1462.65 
Level 3:  1419.9 1462.65 
Successfully inserted key 2617.8
Successfully inserted key 2598.65
Successfully inserted key 2621.35
Successfully inserted key 2624.5
Successfully inserted key 2680.1
Successfully inserted key 2730.5
-----Skiplist for stock 'TITAN'-------
Level 0:  2598.65 2617.8 2621.35 2624.5 2680.1 2730.5 
Level 1:  2624.5 2730.5 
Level 2:  2624.5 2730.5 
Level 3:  2624.5 
Skiplist after intersection: 
Level 0:  
